# Lab06-2 Fancy Softmax Classifier(cross_entropy, one_how, reshape)
#### 2018.09.26(수)

Lab06-1 에서 구현한 Softmax Classifier에 대해서, 
Corss_entropy, one_hot, reshape을 이용하여 더욱 Fancy하게 구현해 보기.

## Softmax_corss_entropy_with_logits
~~~
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)
~~~

~~~
## 1. Cross entropy cost/Loss##
cost = tf.reduece_mean(-tf.reduece_sum(Y * tf.log(hypothesis),axis = 1))
이때, y 는 one-hot으로 주어짐

## 2. Cross entropy cost/Loss ##
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y_one_hot) # logits를 넣는다는걸 반드시 기억!
cost = tf.reduce_mean(cost_i)
~~~

In [2]:
import tensorflow as tf
import numpy as np

In [5]:
# Predicting animal type based based on various features
xy = np.loadtxt('data-04-zoo.csv',delimiter = ',', dtype = np.float)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]
# 이때. Y_data의 raw data를 보면 0~ 6사이의 값이다-> 아직 one-hot encoding이 되어있지 않다.

In [13]:
nb_classes = 7 # 0 ~6의 숫자.
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6 ,  shape = (?,1)
Y_one_hot = tf.one_hot(Y, nb_classes) # one hot shape = (?, 1, 7)
Y_one_hot = tf.reshape(Y_one_hot, [-1,nb_classes]) # shape = (?,7)

# 여기서 주의할 것은, 만약 rank N으로 index를 하여 주게되면 rank N+1 로 output이 나오게 된다.
# 즉, 새로운 axis가 발생함
## If the input inices is rank N, the output will have rank N+1, the new axis is created at dimension axis ( default : the new axis is appended at the end )
# 예를 들면, 2차원 -> [[0],[3]]을 ont_hot 해주면, [[[1,0,0,0,0,0,0],[0,0,0,1,0,0,0]]]으로 output으로 나온다.
# 이를 다시 처리해주기 위해서, reshape을 해준다 -> 우리가 원하는 모양인, [[1,0,0,0,0,0,0],[0,0,0,1,0,0,0]] 으로 바꿔주기 위해서!
# reshape에 -1을 넣는다는건, 알아서 바꾸라는 소리. 여기서 -1은 데이터의 개수가 될 것.

## 전체 코드


In [16]:
# Predicting animal type based based on various features
xy = np.loadtxt('data-04-zoo.csv',delimiter = ',', dtype = np.float)
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

nb_classes = 7 # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1]) # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])

W = tf.Variable(tf.random_normal([16,nb_classes]),name = "weight")
b = tf.Variable(tf.random_normal([nb_classes]),name = "bias")

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits),dim)

logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/Loss
cost_i = tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = Y_one_hot)

cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [19]:
prediction = tf.argmax(hypothesis, 1) # 가장 높은 hypothesis를 가진 값의 index!
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot,1)) # 예측값과, 실제값이 맞으면 1 다르면 0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost,accuracy], feed_dict = {
                X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step,loss,acc))
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict = {X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p==int(y),p, int(y)))
        


Step:     0	Loss: 4.043	Acc: 39.60%
Step:   100	Loss: 0.651	Acc: 87.13%
Step:   200	Loss: 0.412	Acc: 88.12%
Step:   300	Loss: 0.303	Acc: 89.11%
Step:   400	Loss: 0.237	Acc: 91.09%
Step:   500	Loss: 0.193	Acc: 95.05%
Step:   600	Loss: 0.163	Acc: 99.01%
Step:   700	Loss: 0.140	Acc: 99.01%
Step:   800	Loss: 0.122	Acc: 99.01%
Step:   900	Loss: 0.109	Acc: 99.01%
Step:  1000	Loss: 0.098	Acc: 100.00%
Step:  1100	Loss: 0.090	Acc: 100.00%
Step:  1200	Loss: 0.082	Acc: 100.00%
Step:  1300	Loss: 0.076	Acc: 100.00%
Step:  1400	Loss: 0.071	Acc: 100.00%
Step:  1500	Loss: 0.066	Acc: 100.00%
Step:  1600	Loss: 0.062	Acc: 100.00%
Step:  1700	Loss: 0.059	Acc: 100.00%
Step:  1800	Loss: 0.056	Acc: 100.00%
Step:  1900	Loss: 0.053	Acc: 100.00%
Step:  2000	Loss: 0.051	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Predictio